<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [5]:
import io
import os

from google.cloud import speech_v1p1beta1 as speech

client = speech.SpeechClient()

stream_file = os.path.join(os.getcwd(), 'audio_file', 'bong_interview_YTN_16000.wav')    # jupyter notebook korean file_name error

with io.open(stream_file, 'rb') as audio_file:
        content = audio_file.read()

# In practice, stream should be a generator yielding chunks of audio data.
stream = [content]

requests = (speech.types.StreamingRecognizeRequest(audio_content=chunk)
            for chunk in stream)

config = speech.types.RecognitionConfig(
    encoding=speech.enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='ko-KR',
    enable_speaker_diarization=True,
    diarization_speaker_count=3,
    model='default')

streaming_config = speech.types.StreamingRecognitionConfig(config=config)

# streaming_recognize returns a generator.
responses = client.streaming_recognize(streaming_config, requests)

In [6]:
type(responses)

google.api_core.grpc_helpers._StreamingResponseIterator

In [7]:
for response in responses:
    # Once the transcription has settled, the first result will contain the
    # is_final result. The other results will be for subsequent portions of
    # the audio.

    for result in response.results:
        print('Finished: {}'.format(result.is_final))
        print('Stability: {}'.format(result.stability))
        alternatives = result.alternatives
        # The alternatives are ordered from most likely to least.
        for alternative in alternatives:
            print('Confidence: {}'.format(alternative.confidence))
            print(u'Transcript: {}'.format(alternative.transcript))
            
            for words in alternative.words:
                print(words.speaker_tag, end=" ")
            print()

Finished: True
Stability: 0.0
Confidence: 0.8917887806892395
Transcript: 출장을 또 고생했지만 아직 팀장과의 면담은 남아있는 상태지만 이번에 깜찍 성에 대해서 좀 소개를 해 주죠네 한국영화가 100주년을 기념하는 해인데요 올해가 겹경사를 맞았어요 우리나라 영화 중에 유일하게 칸 영화제 경쟁부문에 진출한 영화 기생충이 최고상인 황금종려상을 받은 건데요 그 역사적인 현장에 있었기 때문에 현장 분위기와 함께 보고들은 것 좀 낱낱이 설명해 드리려고 합니다 시상식대상 성을 해 달라는 요청을 받았기 때문에 약간 기대하는 분위기가 있었을 것 같은데 어땠어요 시상식에 참석해 달라는 말이 곧 다시 꺼내 들었다라는 말과 똑같거든요 칸영화제에서는 이게 우리나라의 같은 경우는 시상식에 참석한다고 해서 모든 이들에게 상을주지 많잖아요네 칸 영화제는 시험 시계 좀 참석해 달라 이렇게 얘기를 하면 나도 하나 받겠구나
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
Finished: True
Stability: 0.0
Confidence: 0.9157878160476685
Transcript: 기대를 해도 되는 그런 또 독특한 점은 관례가 있어요 송강호 씨한테 제가 직접 들어보니까 정오부터 1시 사이에 연락이 온다고 하는데 12시 41분 쯤에 연락이 없다고 해요 40분 동안이 본인에게 굉장히 피말리는 순간이었다 이렇게 얘기해도 했었고요 그렇기 때문에 일단 그렇게 알고 나서는 어떤 이제 상을 받을 것인가 그거에 대해서 좀 많이 이제 기자들도 그렇고 많이 집중을 하게 되었던 거 같아요 황금종려상으로 사실을

In [4]:
# speaker_tag가 존재하지만, 모두 1(한명)으로 되어 있음 => 화자 분리 안됨
# for response in responses:
#     for result in response.results:
#         print(result)